# `Промышленное машинное обучение на Spark`
## `Занятие 03: Основы Spark`

### `Находнов Максим (nakhodnov17@gmail.com)`
#### `Москва, 2023`

О чём можно узнать из этого ноутбука:

* DataFrame и SQL API
* Базовые операции в Spark

### `Монтируем диск для хранения данных`

In [1]:
try:
    from google.colab import drive
    drive.mount('/content/drive')
except: pass

In [2]:
! pip3 install -q pyspark pyarrow kaggle parquet-tools


[notice] A new release of pip available: 22.3.1 -> 23.1.2
[notice] To update, run: python3.10 -m pip install --upgrade pip


In [3]:
from pyspark.sql import SparkSession
from pyspark import SparkConf, SparkContext

# Создаём конфигурационный класс с параметрами подключения к серверу
conf = (
    SparkConf()
        # Указываем порт на котором будет располагаться UI
        .set('spark.ui.port', '4050')
        # Указываем URL master ноды Spark кластера
        # Можно использовать local mode, указав `local[<number_cores>]`
        # В таком случае вся обработка будет происходить на текущем компьютере
        # При этом, это может давать преимущество ввиду наличия параллелизма по ядрам компьютера
        .setMaster('local[*]')
        # Если нужно подключиться к "реальному" кластеру то нужно указать URL `spark://<master-node-url:master-node-url>`. Например:
        # .setMaster('spark://localhost:7077')
)
# Создаём точку доступа на кластер. Позволят использовать RDD API
sc = SparkContext(conf=conf)
# Точка доступа для использования DataFrame API
spark = SparkSession(sc)

# По завершении программы нужно обязательно выполнить остановку подключения для освобождения занятых ресурсов
# sc.stop()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/05/03 06:55:15 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


### `Магия для просмотра Spark UI`

При работе в Google Colab/Kaggle можно использовать прокси для доступа к веб-интерфейсу Spark. Для установки и запуска прокси выполните команды ниже:

In [ ]:
! wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
! unzip ngrok-stable-linux-amd64.zip

--2023-01-28 10:00:06--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 52.202.168.65, 54.161.241.46, 54.237.133.81, ...
Connecting to bin.equinox.io (bin.equinox.io)|52.202.168.65|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13832437 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]  13.19M  53.8MB/s    in 0.2s    

2023-01-28 10:00:07 (53.8 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [13832437/13832437]

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   


In [ ]:
get_ipython().system_raw('./ngrok authtoken <token from https://dashboard.ngrok.com/get-started/your-authtoken>')
get_ipython().system_raw('./ngrok http 4050 &')

In [ ]:
! curl -s http://localhost:4040/api/tunnels | python3 -c "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

http://baba-34-74-244-185.ngrok.io


### `Загрузка данных`

Предварительно нужно скачать `kaggle.json` из [настроек аккаунта Kaggle](https://www.kaggle.com/settings). Положите его в папку `~./.kaggle`. На Linux/MacOS это можно сделать следующим образом:

In [4]:
! mkdir ~/.kaggle/
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

mkdir: /Users/nakhodnov/.kaggle/: File exists


В этом проекте нужно работать с данными для предсказания спроса: [M5 Forecasting](https://www.kaggle.com/competitions/m5-forecasting-accuracy/data).

In [5]:
# Используйте путь на Google Drive при работе в Google Colab
path = '/content/drive/MyDrive/m5-forecasting-accuracy'
# Или локальный путь
path = './m5-forecasting-accuracy'

In [6]:
! kaggle competitions download -c m5-forecasting-accuracy

import zipfile
with zipfile.ZipFile('./m5-forecasting-accuracy.zip', 'r') as zip_ref:
    zip_ref.extractall(path)

 90%|██████████████████████████████████    | 41.0M/45.8M [00:02<00:00, 28.1MB/s]
100%|██████████████████████████████████████| 45.8M/45.8M [00:02<00:00, 22.6MB/s]


In [7]:
%ls $path

calendar.csv                sample_submission.csv
sales_train_evaluation.csv  sell_prices.csv
sales_train_validation.csv


In [8]:
# Зададим пути к файлам из датасета
file_calendar = f"{path}/calendar.csv"
file_validation = f"{path}/sales_train_validation.csv"
file_evaluation = f"{path}/sales_train_evaluation.csv"
file_prices = f"{path}/sell_prices.csv"

# Формат данных — CSV
file_type = "csv"
# Зададим параметры, как интерпретировать загруженные данные
# Определять типы колонок автоматически
infer_schema = "true"
# Интерпретируем первую строку в файле, как названия колонок
first_row_is_header = "true"
# Задаём разделитель между значениями колонок
delimiter = ","

df_validation = (
    spark.read.format(file_type)
      .option("inferSchema", infer_schema)
      .option("header", first_row_is_header)
      .option("sep", delimiter)
      .load(file_validation)
    # Также, можно указывать пути в hdfs или базы данных, например, Hive
#       .load('hdfs:///path_to_data/...')
)

df_evaluation = (
    spark.read.format(file_type)
      .option("inferSchema", infer_schema)
      .option("header", first_row_is_header)
      .option("sep", delimiter)
      .load(file_evaluation)
)
df_prices = (
    spark.read.format(file_type)
      .option("inferSchema", infer_schema)
      .option("header", first_row_is_header)
      .option("sep", delimiter)
      .load(file_prices)
)

# Возьмём первые 10 строк pyspark.sql.dataframe.DataFrame
# И выполним action для преобразования в pandas.DataFrame
df_validation.limit(10).toPandas()

23/05/03 06:55:38 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,3,0,1,1,1,3,0,1,1
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,1,2,1,1,1,0,1,1,1
3,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,0,5,4,1,0,1,3,7,2
4,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,1,1,0,1,1,2,2,2,4
5,HOBBIES_1_006_CA_1_validation,HOBBIES_1_006,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,1,0,1,0,0,0,2,0,0
6,HOBBIES_1_007_CA_1_validation,HOBBIES_1_007,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,0,1,0,1,0,0,1,1
7,HOBBIES_1_008_CA_1_validation,HOBBIES_1_008,HOBBIES_1,HOBBIES,CA_1,CA,12,15,0,0,...,0,0,1,37,3,4,6,3,2,1
8,HOBBIES_1_009_CA_1_validation,HOBBIES_1_009,HOBBIES_1,HOBBIES,CA_1,CA,2,0,7,3,...,0,0,1,1,6,0,0,0,0,0
9,HOBBIES_1_010_CA_1_validation,HOBBIES_1_010,HOBBIES_1,HOBBIES,CA_1,CA,0,0,1,0,...,1,0,0,0,0,0,0,2,0,2


### `Spark DataFrame API`

* [Quickstart](https://spark.apache.org/docs/latest/api/python/getting_started/quickstart_df.html)
* [Документация](https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/dataframe.html)

In [9]:
emp_data = [
    (1, 'Smith', 10),
    (2, 'Rose', 20),
    (3, 'Williams', 10),
    (4, 'Jones', 30),
    (5, 'Jones', None),
]
emp_columns = ['emp_id', 'name', 'dept_id']

emp_df = spark.createDataFrame(emp_data, emp_columns)
emp_df

DataFrame[emp_id: bigint, name: string, dept_id: bigint]

In [10]:
type(emp_df)

pyspark.sql.dataframe.DataFrame

Вывод DataFrame не показывает его содержимое, так как оно ещё не было вычислено, так как вычисления в Spark происходят только в момент вызова action.

Примеры action:
* `count()` — подсчитывает число строк в DataFrame
* `toPandas()` — преобразует Spark DataFrame в pandas DataFrame
* `collect()` — выполняет вычисление текущего Spark DataFrame и возвращает результат
* `show()` — `collect()` + pretty print результата

In [11]:
emp_df.show()

+------+--------+-------+
|emp_id|    name|dept_id|
+------+--------+-------+
|     1|   Smith|     10|
|     2|    Rose|     20|
|     3|Williams|     10|
|     4|   Jones|     30|
|     5|   Jones|   null|
+------+--------+-------+



Базовая информация о данных — названия колонок и их типы:

In [12]:
emp_df.columns, emp_df.schema

(['emp_id', 'name', 'dept_id'],
 StructType([StructField('emp_id', LongType(), True), StructField('name', StringType(), True), StructField('dept_id', LongType(), True)]))

Многие методы дублируются по аналогии с `pandas.DataFrame`:

In [13]:
emp_df.dropna().show()

+------+--------+-------+
|emp_id|    name|dept_id|
+------+--------+-------+
|     1|   Smith|     10|
|     2|    Rose|     20|
|     3|Williams|     10|
|     4|   Jones|     30|
+------+--------+-------+



DataFrame состоит из колонок. Получение колонки возможно через атрибуты или через индексацию:

In [14]:
emp_df.name, emp_df['name']

(Column<'name'>, Column<'name'>)

Вместо самих значений колонок, в соответствии с принципом "ленивых" вычислений, возвращаются ссылки на них. Такие колонки могут участвовать в символьных вычислениях. Например, к ним можно применять арифметические, булевы операции.

In [15]:
column_expr = (emp_df.dept_id - 20) / 10 > emp_df.emp_id
column_expr

Column<'(((dept_id - 20) / 10) > emp_id)'>

Полученные **колоночные выражения** (**column expression**) можно вычислять:

In [16]:
emp_df.select(column_expr).show()

+--------------------------------+
|(((dept_id - 20) / 10) > emp_id)|
+--------------------------------+
|                           false|
|                           false|
|                           false|
|                           false|
|                            null|
+--------------------------------+



Колонку можно переименовать:

In [17]:
emp_df.select((emp_df.dept_id ** 2).alias('dept_id squared')).show()

+---------------+
|dept_id squared|
+---------------+
|          100.0|
|          400.0|
|          100.0|
|          900.0|
|           null|
+---------------+



Для DataFrame доступны SQL подобные операции, например, `join`:

In [18]:
dept_data = [
    ('Finance', 10),
    ('Marketing', 20),
    ('Sales', 30),
    ('IT', 40),
]
dept_columns = ['dept_name', 'dept_id']

dept_df = spark.createDataFrame(dept_data, dept_columns)
dept_df.show()

+---------+-------+
|dept_name|dept_id|
+---------+-------+
|  Finance|     10|
|Marketing|     20|
|    Sales|     30|
|       IT|     40|
+---------+-------+



In [19]:
emp_df.join(dept_df, how='inner', on=['dept_id']).show()

+-------+------+--------+---------+
|dept_id|emp_id|    name|dept_name|
+-------+------+--------+---------+
|     10|     1|   Smith|  Finance|
|     10|     3|Williams|  Finance|
|     20|     2|    Rose|Marketing|
|     30|     4|   Jones|    Sales|
+-------+------+--------+---------+



In [20]:
emp_df.join(dept_df, how='outer', on=['dept_id']).show()

+-------+------+--------+---------+
|dept_id|emp_id|    name|dept_name|
+-------+------+--------+---------+
|   null|     5|   Jones|     null|
|     10|     3|Williams|  Finance|
|     10|     1|   Smith|  Finance|
|     20|     2|    Rose|Marketing|
|     30|     4|   Jones|    Sales|
|     40|  null|    null|       IT|
+-------+------+--------+---------+



Также, доступна фильтрация и сортировка:

In [21]:
(
    emp_df
      .join(dept_df, how='outer', on=['dept_id'])
      # Обратите внимание на колоночное выражение в фильтре
      .where((emp_df['name'] == 'Smith') | (emp_df['name'] == 'Rose'))
      .sort('dept_id')
      .show()
)

+-------+------+-----+---------+
|dept_id|emp_id| name|dept_name|
+-------+------+-----+---------+
|     10|     1|Smith|  Finance|
|     20|     2| Rose|Marketing|
+-------+------+-----+---------+



Работа с колонками обычно выполняется через колоночные выражения. Их можно использовать, например, для выполнения join:

In [22]:
emp_columns_renamed = ['emp_id', 'name', 'emp_dept_id']

emp_renamed_df = spark.createDataFrame(emp_data, emp_columns_renamed)
emp_renamed_df.show()

+------+--------+-----------+
|emp_id|    name|emp_dept_id|
+------+--------+-----------+
|     1|   Smith|         10|
|     2|    Rose|         20|
|     3|Williams|         10|
|     4|   Jones|         30|
|     5|   Jones|       null|
+------+--------+-----------+



In [23]:
emp_renamed_df.join(
    dept_df, emp_renamed_df.emp_dept_id == dept_df.dept_id,  how='inner'
).show()

+------+--------+-----------+---------+-------+
|emp_id|    name|emp_dept_id|dept_name|dept_id|
+------+--------+-----------+---------+-------+
|     1|   Smith|         10|  Finance|     10|
|     3|Williams|         10|  Finance|     10|
|     2|    Rose|         20|Marketing|     20|
|     4|   Jones|         30|    Sales|     30|
+------+--------+-----------+---------+-------+



Переименование колонок также возможно:

In [24]:
(
    emp_renamed_df
      .withColumnRenamed('emp_dept_id', 'dept_id')
      .join(
          dept_df, 'dept_id',  how='inner'
      )
      .show()
)

+-------+------+--------+---------+
|dept_id|emp_id|    name|dept_name|
+-------+------+--------+---------+
|     10|     1|   Smith|  Finance|
|     10|     3|Williams|  Finance|
|     20|     2|    Rose|Marketing|
|     30|     4|   Jones|    Sales|
+-------+------+--------+---------+



Для преобразования колонок в модуле `pyspark.sql.functions` содержится большой набор вспомогательных функций. Например:
* Вспомогательные: `lit`, `col`, ...
* Поэлементные математические функции: `cos`, `sin`, `round`, ...
* Поэлементные функции для работы датами и временем: `dayofmonth`, ...
* Агрегаторы: `sum`, `mean`, ...
* Функции для работы с коллекциями (сложными данными, хранящимся в колонке): `array_sort`, `concat`, ...
* Сортировки: `asc`, ...
* Строковые функции: `concat_ws`, `lower`, `split`, ...
* Оконные функции: `lag`, ...
* Преобразования с пользовательскими функциями: `udf_pandas`, ...

**Всегда перед написанием кода нужно подумать, нет ли уже готовой функции в данном модуле. Использование готовых функции существенно влияет на скорость вычислений.**

In [25]:
import pyspark.sql.functions as F
from pyspark.sql.types import StringType, DateType

Часто для применения функций нужно поменять тип колонки:

In [26]:
emp_with_date = (
    emp_df
        .dropna()
        .withColumn(
            'hire_date', 
            # Конструируем дату в формате yyyy-mm-dd 
            F.concat_ws(
                '-', 
                # Придумываем год
                (1990 + emp_df.dept_id).cast(StringType()),
                # Придумываем месяц
                F.concat(F.lit('0'), emp_df.emp_id.cast(StringType())), 
                # Придумываем день
                emp_df.dept_id.cast(StringType())
            ).cast(DateType())
        )
)
emp_with_date.show()

+------+--------+-------+----------+
|emp_id|    name|dept_id| hire_date|
+------+--------+-------+----------+
|     1|   Smith|     10|2000-01-10|
|     2|    Rose|     20|2010-02-20|
|     3|Williams|     10|2000-03-10|
|     4|   Jones|     30|2020-04-30|
+------+--------+-------+----------+



In [27]:
emp_with_date.select(
    F.acos(emp_with_date.emp_id / 4),
    F.year(emp_with_date.hire_date),
    F.regexp_replace(F.lower(emp_with_date.name), 'smith', 'cмит').alias('processed_name')
).show()

+------------------+---------------+--------------+
|ACOS((emp_id / 4))|year(hire_date)|processed_name|
+------------------+---------------+--------------+
| 1.318116071652818|           2000|          cмит|
|1.0471975511965979|           2010|          rose|
|0.7227342478134157|           2000|      williams|
|               0.0|           2020|         jones|
+------------------+---------------+--------------+



### `Spark SQL API`

In [28]:
add_data = [
    (1, '1523 Main St', 'SFO', 'CA'),
    (2, '3453 Orange St', 'SFO', 'NY'),
    (3, '34 Warner St', 'Jersey', 'NJ'),
    (4, '221 Cavalier St', 'Newark', 'DE'),
    (5, '789 Walnut St', 'Sandiago', 'CA')
]
add_columns = ['emp_id', 'address', 'city', 'state']

add_df = spark.createDataFrame(add_data, add_columns)
add_df.show()

+------+---------------+--------+-----+
|emp_id|        address|    city|state|
+------+---------------+--------+-----+
|     1|   1523 Main St|     SFO|   CA|
|     2| 3453 Orange St|     SFO|   NY|
|     3|   34 Warner St|  Jersey|   NJ|
|     4|221 Cavalier St|  Newark|   DE|
|     5|  789 Walnut St|Sandiago|   CA|
+------+---------------+--------+-----+



Spark позволяет использовать DataFrame в качестве таблиц в регулярных SQL запросах:

In [29]:
emp_df.createOrReplaceTempView('EMP')
dept_df.createOrReplaceTempView('DEPT')
add_df.createOrReplaceTempView('ADD')

In [30]:
spark.sql('''
    select * from EMP e, DEPT d, ADD a
    where e.dept_id == d.dept_id and e.emp_id == a.emp_id
''').show()

+------+--------+-------+---------+-------+------+---------------+------+-----+
|emp_id|    name|dept_id|dept_name|dept_id|emp_id|        address|  city|state|
+------+--------+-------+---------+-------+------+---------------+------+-----+
|     1|   Smith|     10|  Finance|     10|     1|   1523 Main St|   SFO|   CA|
|     2|    Rose|     20|Marketing|     20|     2| 3453 Orange St|   SFO|   NY|
|     3|Williams|     10|  Finance|     10|     3|   34 Warner St|Jersey|   NJ|
|     4|   Jones|     30|    Sales|     30|     4|221 Cavalier St|Newark|   DE|
+------+--------+-------+---------+-------+------+---------------+------+-----+



### `Ещё базовые операции над Spark DataFrame`

In [31]:
data = [
    ('James', 'Sales', 3000),
    ('Michael', 'Sales', 4600),
    ('Robert', 'Sales', 4100),
    ('Maria', 'Finance', 3000),
    ('James', 'Sales', 3000),
    ('Scott', 'Finance', 3300),
    ('Jen', 'Finance', 3900),
    ('Jeff', ' Marketing', 3000),
    ('Kumar', 'Marketing', 2000),
    ('Saif', 'Sales', 4100),
]
columns = ['Name', 'Dept', 'Salary']

df = spark.createDataFrame(data, columns)
df.show()

+-------+----------+------+
|   Name|      Dept|Salary|
+-------+----------+------+
|  James|     Sales|  3000|
|Michael|     Sales|  4600|
| Robert|     Sales|  4100|
|  Maria|   Finance|  3000|
|  James|     Sales|  3000|
|  Scott|   Finance|  3300|
|    Jen|   Finance|  3900|
|   Jeff| Marketing|  3000|
|  Kumar| Marketing|  2000|
|   Saif|     Sales|  4100|
+-------+----------+------+



In [32]:
df.distinct().show()

+-------+----------+------+
|   Name|      Dept|Salary|
+-------+----------+------+
|  James|     Sales|  3000|
|Michael|     Sales|  4600|
| Robert|     Sales|  4100|
|  Maria|   Finance|  3000|
|  Scott|   Finance|  3300|
|    Jen|   Finance|  3900|
|   Jeff| Marketing|  3000|
|  Kumar| Marketing|  2000|
|   Saif|     Sales|  4100|
+-------+----------+------+



In [33]:
df.distinct().count()

9

Также, возможно использовать группировку и агрегаты:

In [34]:
df.groupBy('Dept').sum().collect()

[Row(Dept='Sales', sum(Salary)=18800),
 Row(Dept='Finance', sum(Salary)=10200),
 Row(Dept=' Marketing', sum(Salary)=3000),
 Row(Dept='Marketing', sum(Salary)=2000)]

### `IO операции`

In [35]:
base_statistics = df.select(
    F.min('Salary').alias('min_salary'),
    F.mean('Salary').alias('mean_salary'),
    F.max('Salary').alias('max_salary')
)
# Пока никаких вычислений не произошло
base_statistics

DataFrame[min_salary: bigint, mean_salary: double, max_salary: bigint]

In [36]:
base_statistics.write.csv('./base_statistics.csv', header=True)
base_statistics.write.parquet('./base_statistics.parquet')

In [37]:
%ls ./base_statistics.csv/
%pycat ./base_statistics.csv/part-00000-5f1b82c2-0fe0-4305-8c71-067297fd29b2-c000.csv

_SUCCESS
part-00000-5f1b82c2-0fe0-4305-8c71-067297fd29b2-c000.csv


In [38]:
%ls ./base_statistics.parquet/
! parquet-tools inspect base_statistics.parquet/part-00000-69467b95-0711-4c15-a58c-e6370e0886c9-c000.snappy.parquet

_SUCCESS
part-00000-69467b95-0711-4c15-a58c-e6370e0886c9-c000.snappy.parquet

############ file meta data ############
created_by: parquet-mr version 1.12.2 (build 77e30c8093386ec52c3cfa6c34b7ef3321322c94)
num_columns: 3
num_rows: 1
num_row_groups: 1
format_version: 1.0
serialized_size: 789


############ Columns ############
min_salary
mean_salary
max_salary

############ Column(min_salary) ############
name: min_salary
path: min_salary
max_definition_level: 1
max_repetition_level: 0
physical_type: INT64
logical_type: None
converted_type (legacy): NONE
compression: SNAPPY (space_saved: -5%)

############ Column(mean_salary) ############
name: mean_salary
path: mean_salary
max_definition_level: 1
max_repetition_level: 0
physical_type: DOUBLE
logical_type: None
converted_type (legacy): NONE
compression: SNAPPY (space_saved: -5%)

############ Column(max_salary) ############
name: max_salary
path: max_salary
max_definition_level: 1
max_repetition_level: 0
physical_type: INT64
logical_typ

In [39]:
loaded_df = (
    spark.read
        .format('csv')
        .option("inferSchema", True)
        .option("header", True)
        .option("sep", ',')
        .load('./base_statistics.csv')
)
loaded_df, loaded_df.show()

+----------+-----------+----------+
|min_salary|mean_salary|max_salary|
+----------+-----------+----------+
|      2000|     3400.0|      4600|
+----------+-----------+----------+



(DataFrame[min_salary: int, mean_salary: double, max_salary: int], None)